In [2]:
from alpa_serve.trace import Trace, TraceReplay, report_group_stats
from scipy.stats import entropy
import numpy as np
import time
import queue
import threading
from alpa_serve.simulator.workload import Workload, GammaProcess, UniformMMPP
trace_name = "azure_v2"
trace_dir = "../azure_v2.pkl"

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-20 21:47:25,179	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
trace = Trace(trace_name, trace_dir)

Reading takes: 0.02273392677307129s.


In [4]:
n_model = 5
models = [f"gpt{i}" for i in range(n_model)]
train_start = "13.0.0"
train_end = "13.23.60"
# replays = trace.replay_vanilla(models,
#                                model_mapping_strategy="stripe",
#                                start_time="0.0.0",
#                                end_time="1.0.0")
replays = trace.replay(models,
                               model_mapping_strategy="stripe",
                               arrival_distribution="gamma",
                              start_time=train_start,
                              end_time=train_end,
                              interval_seconds=5400)

Sliced trace stats: #functions: 424, total invocations: 93988, max: 26116, min: 0, avg: 221.67
5 144


In [5]:
ws = []
model_name = 'gpt0'
slo_scale = 1
single_latency = 1000
num_models = 1
slos = [slo_scale * single_latency] * num_models
ws.append(replays[model_name].to_workload(slos[0]))
workload = Workload.merge(*ws)
input_ = "Paris is the capital city of"
args = [(input_, workload.requests[i].model_name,
                 workload.requests[i].slo, float(workload.arrivals[i]),
                 i) for i in range(len((workload)))]

In [ ]:
submit_queue = queue.Queue()
num_submit_threads = 4
submit_threads = [
    threading.Thread(
        target=submit_worker_func, args=(submit_queue)
    ) for _ in range(num_submit_threads)
]
for t in submit_threads:
    t.start()
def submit_request(*args):
    submit_queue.put_nowait(args)
def synchronize():
    submit_queue.join()

def close_copy_threads():
    for _ in range(len(submit_threads)):
        submit_queue.put_nowait(None)
    for t in submit_threads:
        t.join()
    submit_queue.join()
    submit_queue = None
def submit_worker_func(queue):
    while True:
        item = queue.get()
        if item is None:
            queue.task_done()
            return
        # do sth
        queue.task_done()
    

In [10]:
# float(workload.arrivals[i])
# 仿照submit_one(arg):
for i in range(len((workload))):
    start =  time.time() + float(workload.arrivals[i]) - 1123200
    while time.time() < start:
        pass
    time.sleep(1) # submit 
    end = time.time()
    e2e_latency = end - start
    print(e2e_latency)

1.0011398792266846
1.001917839050293
1.0011990070343018
1.001037836074829


KeyboardInterrupt: 

In [13]:
(float(workload.arrivals[-1]) - float(workload.arrivals[0]))

86370.18783562398

In [15]:
len(workload.arrivals)

11399

In [ ]:
train_workload.requests[0].model_name

In [ ]:
[slo_scale * single_latency] * 2

In [ ]:
replays['gpt0'].report_stats()
replays['gpt0'].visualize()

In [ ]:
replays['gpt0'].arrivals

In [ ]:
replays = trace.replay(models,
                       model_mapping_strategy="stripe",
                       start_time="0.0.0",
                       end_time="5.0.0",
                       arrival_distribution="exponential",
                       interval_seconds=3600)

In [1]:
('xxx',) * 2

('xxx', 'xxx')

In [2]:
for block_index in range(12, 23):
    print(block_index)

12
13
14
15
16
17
18
19
20
21
22
